# Exploring a Single Subject

The purpose of this notebook is take all subjects and segment them assuming they are clean.

The researcher/student is not intended to make changes here.

In [ ]:
import mne
import pandas as pd
import pylossless as ll
import glob
import json
mne.viz.set_browser_backend('qt')

# EEG Configuration
filter_freqs = (1.0, 30.0) # Filter data from (low_freq, high_freq)
channel_interest = ['all']
epoch_tmin = -0.1 # Baseline to stimulus onset in seconds, negative implies before stimulus
epoch_tmax = 1.0 # Duration after stimulus of epoch
reject_criteria = {'eeg': 200e-6} # 200 microvolts peak to peak
conditions = ['VO21', 'VO22', 'VO23', 'VO24', 'VO25']

In [ ]:
def clean_and_epoch(subject_id):
    project_path = '/media/tyler/PathstoneProject/PathstoneYAC'
    raw_path = f'{project_path}/sub-YAC{subject_id}/eeg/sub-YAC{subject_id}_task-AttnCtrl_eeg.edf'
    derivative_path = f'{project_path}/derivatives/pylossless/sub-YAC{subject_id}/eeg/sub-YAC{subject_id}_task-afd_eeg.edf'

    ll_state = ll.LosslessPipeline()
    ll_state = ll_state.load_ll_derivative(derivative_path)
    ll_state.raw.info['bads'] = ll_state.flags['ch'].get_flagged()
    ll_state.ica2.exclude = [index for index,comp in ll_state.flags['ic'].iterrows() if comp['ic_type'] in ['eog', 'ecg', 'muscle', 'line_noise', 'channel_noise']]

    cleaned_state = ll_state.raw.copy()
    cleaned_state.load_data()
    ll_state.ica2.apply(cleaned_state)
    cleaned_state = cleaned_state.interpolate_bads()
    cleaned_state = cleaned_state.filter(l_freq=filter_freqs[0], h_freq=filter_freqs[1])

    raw_data = mne.io.read_raw_edf(f'sub-YAC{subject_id}/eeg/sub-YAC{subject_id}_task-AttnCtrl_eeg.edf')
    base_events, event_dict = mne.events_from_annotations(raw_data)

    event_dict['VO24/combine'] = event_dict.pop('VO24')
    event_dict['VO25/combine'] = event_dict.pop('VO25')
    
    cleaned_state = cleaned_state.set_eeg_reference(ref_channels="average")

    epochs = mne.Epochs(
        cleaned_state,
        base_events,
        picks=channel_interest,
        event_id=event_dict,
        tmin=epoch_tmin,
        tmax=epoch_tmax,
        preload=True,
        reject=reject_criteria,
        event_repeated='merge',
    )
    # Trim down to only selected event types
    epochs = epochs[conditions]
    # Print the event counts
    event_counts = {label:len(epochs[label]) for label in conditions}
    event_counts['subject'] = subject_id
    epochs.save(f'derivatives/segmented/yac_{subject_id}_pylqcr_eeg.fif', overwrite=True)
    return event_counts

In [ ]:
subject_ids = [item[7:] for item in sorted(glob.glob('sub-*'))]

subject_report = []
for subject in subject_ids:
    print('Working on subject: ', subject)
    out_str = clean_and_epoch(subject)
    subject_report.append(out_str)
df = pd.DataFrame(subject_report)
df = df.set_index('subject')
df.to_csv('derivatives/segmented/event_report.csv')
df